# Preparing datasets: Mar'24
This notebook is meant to generate a new dataset (only for March 2024) based on *Citibike Trip data* that can be used to determine **bike rentals** based on a <u>fixed time resolution</u>. 

## Preparing Trip data

In [1]:
import pandas as pd
citi_mar_24 = pd.read_csv("C:/Users/singh/Desktop/TUD (All Semesters)/Courses - Semester 5 (TU Dresden)/Research Task - Spatial Modelling/Datasets/Citibike Trip Data/202403-citibike-tripdata.csv")
citi_mar_24.head()

C:\Users\singh\AppData\Local\Temp\ipykernel_17924\484946610.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  citi_mar_24 = pd.read_csv("C:/Users/singh/Desktop/TUD (All Semesters)/Courses - Semester 5 (TU Dresden)/Research Task - Spatial Modelling/Datasets/Citibike Trip Data/202403-citibike-tripdata.csv")


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,62021B31AF42943E,electric_bike,2024-03-13 15:57:41.800,2024-03-13 16:07:09.853,Forsyth St & Grand St,5382.07,Front St & Jay St,4895.03,40.717763,-73.993166,40.702461,-73.986842,member
1,EC7BE9D296FFD072,electric_bike,2024-03-16 10:25:46.114,2024-03-16 10:30:21.554,E 12 St & 3 Ave,5788.12,Mott St & Prince St,5561.04,40.732456,-73.988554,40.723180,-73.994800,member
2,EC85C0EEC95157BB,classic_bike,2024-03-20 19:20:49.818,2024-03-20 19:28:00.165,E 12 St & 3 Ave,5788.12,Mott St & Prince St,5561.04,40.732233,-73.988900,40.723180,-73.994800,member
3,9DDE9AF5606B4E0F,classic_bike,2024-03-13 20:31:12.599,2024-03-13 20:40:31.209,6 Ave & W 34 St,6364.1,E 25 St & 1 Ave,6004.07,40.749640,-73.988050,40.738177,-73.977387,member
4,E4446F457328C5FE,electric_bike,2024-03-16 10:50:11.535,2024-03-16 10:53:02.451,Cleveland Pl & Spring St,5492.05,Mott St & Prince St,5561.04,40.721995,-73.997344,40.723180,-73.994800,member


In [2]:
# The dataset needs timeline trimming - exclude Feb data
print(min(citi_mar_24["started_at"]), max(citi_mar_24["started_at"]))

2024-02-29 00:20:27.570 2024-03-31 23:57:16.025


In [3]:
# There are NaN values in the dataset
sum(citi_mar_24["start_station_name"].isna())

2796

In [4]:
# If the dataset is filtered for 'classic bikes' all starting stations should be visible
sum(citi_mar_24[citi_mar_24["rideable_type"] == "classic_bike"]["start_station_name"].isna())

0

In [5]:
# filtering the dataset
cbikes = citi_mar_24[citi_mar_24["rideable_type"] == "classic_bike"]
cbikes["rideable_type"].unique()

array(['classic_bike'], dtype=object)

In [6]:
# reducing columns
cbikes = cbikes[["ride_id", "started_at", "ended_at", "start_station_name", "start_station_id", "end_station_name", "end_station_id", "start_lat", "start_lng", "end_lat", "end_lng"]]

# ride_id can be used to count trips
print(len(cbikes), len(cbikes["ride_id"].unique()))

938597 938597


Instead of relying on station_ids provided by **Citibike**, the manually constructed IDs can be entered in dataset. This will make it easy for referencing and connecting *rentals* table below with this original filtered dataset.<br><br>
<u>Note</u>: Original filtered dataset means data for classic bikes where NaN values and redundant columns are removed i.e. **cbikes** dataframe.

## Creating Rentals Template
The new template should contain stations with their location and time information included. Each row represents <u> one unit of chosen time resolution</u>.
<br><br>
**NOTE:** Station ID needs to be retained because there are some stations with the same name but slightly different locations!
<br><br>
In the dataset, some IDs are stored as text and others are stored as numbers. Converting th data type to *string* does not work because **\n** and some other text is being added automatically.
<br><br>
**Solution:**<br>
The stations can be manually given unique ids for the purpose of indentification.

In [7]:
# Creating the new dataset
rentals = pd.DataFrame()
rentals[["name", "lat", "lng"]] = cbikes[["start_station_name", "start_lat", "start_lng"]].drop_duplicates()
rentals = rentals.sort_values(by = ["name"], ignore_index=True)

# Adding IDs manually
rentals["station_id"] = range(len(rentals))

# Verifying the number of stations
print(len(cbikes[["start_station_name", "start_lat", "start_lng"]].drop_duplicates()), len(rentals["station_id"]))

2127 2127


## Reformatting Tripdata
Making some changes to the trip data before exporting.

In [8]:
# Copying cbikes df
cbikes_altered = cbikes.copy()

# Merging the two dfs
new_df = pd.merge(
    left=cbikes_altered, 
    right=rentals,
    how='left',
    left_on=['start_station_name', 'start_lat', 'start_lng'],
    right_on=['name', 'lat', 'lng'],
)

# Preserving bike trip info only with the starting station info
cbikes_altered = new_df[["ride_id", "started_at", "start_station_name", "station_id", "start_lat", "start_lng"]]

# Checking validity
print(len(cbikes_altered["station_id"].unique()))
cbikes_altered.head()

2127


,ride_id,started_at,start_station_name,station_id,start_lat,start_lng
0,EC85C0EEC95157BB,2024-03-20 19:20:49.818,E 12 St & 3 Ave,918,40.732233,-73.988900
1,9DDE9AF5606B4E0F,2024-03-13 20:31:12.599,6 Ave & W 34 St,364,40.749640,-73.988050
2,0132F3650F73AA65,2024-03-15 13:37:19.823,6 Ave & W 34 St,364,40.749640,-73.988050
3,B47552E7DA344D68,2024-03-04 19:41:29.718,Lexington Ave & E 120 St,1400,40.801307,-73.939817
4,7BE030A97D400E4A,2024-03-03 14:59:36.914,W 17 St & 7 Ave,1924,40.740564,-73.998526


In [9]:
# exporting to 'C:\Users\singh\Desktop\TUD (All Semesters)\Courses - Semester 5 (TU Dresden)\Research Task - Spatial Modelling\Code'
cbikes_altered.to_csv("Tripdata_with_manual_ids_mar.csv", index=False)

## Defining time resolution
Here time units are defined that will later be merged with the *rentals* template.

In [10]:
# importing the data
import pandas as pd
cbikes_altered = pd.read_csv("C:/Users/singh/Desktop/TUD (All Semesters)/Courses - Semester 5 (TU Dresden)/Research Task - Spatial Modelling/Code/Tripdata_with_manual_ids_mar.csv")

# defining the 7 time units for a given date
time_res = ['08:00:00.000', '10:00:00.000', '12:00:00.000', '14:00:00.000', '16:00:00.000', '18:00:00.000', '20:00:00.000']

# repeating time units 31 times - till 31 Mar
time_res = time_res*31

# creating dates
dates_included = []
while len(dates_included) < 31:
    dates_included.append('2024-03-01')

# days list
nums = ['01', '02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29', '30', '31']

# correcting days
for i in range(len(nums)):
    dates_included[i] = dates_included[i][:-2] + nums[i]

dates_included[:5]

['2024-03-01', '2024-03-02', '2024-03-03', '2024-03-04', '2024-03-05']

Now, we are supposed to combine the date and the time information together. Each day has 7 time units, so each day must be repeated 7 times to accodomate the time resolution.

In [11]:
# repeating each element 7 times
dates_included = [element for element in dates_included for _ in range(7)]

# Adding time dimension
date_time = dates_included.copy()

for t in range(0,len(dates_included)):
    date_time[t] = dates_included[t] + ' ' + time_res[t]

date_time[:7]

['2024-03-01 08:00:00.000',
 '2024-03-01 10:00:00.000',
 '2024-03-01 12:00:00.000',
 '2024-03-01 14:00:00.000',
 '2024-03-01 16:00:00.000',
 '2024-03-01 18:00:00.000',
 '2024-03-01 20:00:00.000']

## Adding time dimensionality to the *rentals* template
A time resolution needs to be added so that number of rentals can be calculated in terms of that time unit, per station.

In [12]:
# rows needed for each station
len(date_time)

217

In [13]:
# duplicating each station 217 times
rentals = rentals.loc[rentals.index.repeat(217)].reset_index(drop=True)

# Repeating date_time: 2127 unique stations
date_time = date_time*2127
rentals["datetime"] = date_time
rentals.head()

,name,lat,lng,station_id,datetime
0,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 08:00:00.000
1,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 10:00:00.000
2,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 12:00:00.000
3,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 14:00:00.000
4,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 16:00:00.000


## Calculating Demand
The rentals are calculated for every time unit, per day per station. The dataset is being prepared only until 31 March 2024.

In [14]:
# importing
tripdata = pd.read_csv("C:/Users/singh/Desktop/TUD (All Semesters)/Courses - Semester 5 (TU Dresden)/Research Task - Spatial Modelling/Code/Tripdata_with_manual_ids_mar.csv")

# adding a rentals column 
rentals['#_rentals'] = 0

rentals.head()

,name,lat,lng,station_id,datetime,#_rentals
0,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 08:00:00.000,0
1,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 10:00:00.000,0
2,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 12:00:00.000,0
3,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 14:00:00.000,0
4,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 16:00:00.000,0


In [15]:
# Updating the rentals dataset
rentals['year'] = pd.Series([x.date().year for x in pd.to_datetime(rentals["datetime"])])
rentals['month'] = pd.Series([x.date().month for x in pd.to_datetime(rentals["datetime"])])
rentals['day'] = pd.Series([x.date().day for x in pd.to_datetime(rentals["datetime"])])

rentals['hour'] = pd.Series([x.time().hour for x in pd.to_datetime(rentals["datetime"])])
rentals['minute'] = pd.Series([x.time().minute for x in pd.to_datetime(rentals["datetime"])])
rentals['second'] = pd.Series([x.time().second for x in pd.to_datetime(rentals["datetime"])])
rentals['microsecond'] = pd.Series([x.time().microsecond for x in pd.to_datetime(rentals["datetime"])])

# Updating the trips dataset
tripdata['started_at_year'] = pd.Series([x.date().year for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_month'] = pd.Series([x.date().month for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_day'] = pd.Series([x.date().day for x in pd.to_datetime(tripdata["started_at"])])

tripdata['started_at_hour'] = pd.Series([x.time().hour for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_minute'] = pd.Series([x.time().minute for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_second'] = pd.Series([x.time().second for x in pd.to_datetime(tripdata["started_at"])])
tripdata['started_at_microsecond'] = pd.Series([x.time().microsecond for x in pd.to_datetime(tripdata["started_at"])])

In [16]:
# The tripdata has some misleading Feb data that should be removed
min(tripdata["started_at_month"])

2

In [17]:
# After Correction
tripdata = tripdata[tripdata["started_at_month"] > 2]
min(tripdata["started_at_month"])

3

In [19]:
# Calculating demand from tripdata for Mar 2024
for k in range(1, 32):
    
    # determining demand for 2024; 08-10hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 8) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] < 10) & (tripdata["started_at_hour"] >= 8) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 10-12hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 10) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 10) & (tripdata["started_at_hour"] < 12) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 12-14hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 12) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 12) & (tripdata["started_at_hour"] < 14) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 14-16hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 14) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 14) & (tripdata["started_at_hour"] < 16) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 16-18hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 16) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 16) & (tripdata["started_at_hour"] < 18) & (tripdata["station_id"] == i)])
        
    # determining demand for 2024; 18-20hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 18) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 18) & (tripdata["started_at_hour"] < 20) & (tripdata["station_id"] == i)])
    
    # determining demand for 2024; 20-22hrs
    for i in rentals["station_id"].unique():
        idx = (rentals["station_id"] == i) & (rentals["year"] == 2024) & (rentals["hour"] == 20) & (rentals["day"] == k)
        rentals.loc[idx, "#_rentals"] = len(tripdata[(tripdata["started_at_year"] == 2024) & (tripdata["started_at_day"] == k) & (tripdata["started_at_hour"] >= 20) & (tripdata["started_at_hour"] < 22) & (tripdata["station_id"] == i)])

In [21]:
# total trips on mar 1 2024
print(len(tripdata[(tripdata["started_at_year"] == 2024)&(tripdata["started_at_day"] == 1)&(tripdata["started_at_hour"] >= 8)&(tripdata["started_at_hour"] < 22)]))

# rentals recorded for mar 1 2024 
sum(rentals.loc[(rentals["year"] == 2024) & (rentals["day"] == 1),"#_rentals"])

24907


24907

Hence, the calculation is correct!

In [22]:
# Created dataset
rentals_final = rentals.copy()

# removing unneeded columns
rentals_final.drop(columns=['minute','second','microsecond'], inplace=True)

# final look
rentals_final.head()

,name,lat,lng,station_id,datetime,#_rentals,year,month,day,hour
0,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 08:00:00.000,3,2024,3,1,8
1,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 10:00:00.000,1,2024,3,1,10
2,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 12:00:00.000,1,2024,3,1,12
3,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 14:00:00.000,0,2024,3,1,14
4,1 Ave & E 110 St,40.792327,-73.9383,0,2024-03-01 16:00:00.000,0,2024,3,1,16


In [23]:
# exporting
rentals_final.to_csv("rentals_with_demand_new_time_units_mar.csv", index=False)